# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import warnings
from sklearn.dummy import DummyClassifier
warnings.filterwarnings('ignore')

In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# читаем файл
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
# смотрим сверху
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [5]:
# видим дисбаланс классов
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# лемматизируем
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


lemmatizer = WordNetLemmatizer()

df['text'] = df['text'].apply(lambda text : 
                             " ".join(
                                 re.sub(r'[^a-zA-Z]', ' ',
                                        ''.join(lemmatizer.lemmatize(text.lower(), 
                                                                     get_wordnet_pos(text.lower())))).split()))

df.head()


,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


## Обучение

In [8]:
# выделяем признаки и целевую переменную
X = df.drop(['toxic'], axis=1)
y = df['toxic']

In [9]:
# разбиваем на выборки
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.4, 
                                                    random_state=12345)

In [10]:
# подгружаем стоп слова
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# векторизуем
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [12]:
X_train = count_tf_idf.fit_transform(X_train['text'].values)
X_test = count_tf_idf.transform(X_test['text'].values)

**LogisticRegression**

In [13]:
%%time

param_grid = {
      'C': range(1,11)}


estimator = LogisticRegression(random_state=12345, 
                               solver='liblinear', 
                               verbose = False, 
                               class_weight='balanced')

model_lr = RandomizedSearchCV(estimator, 
                          param_grid,
                          verbose = False,
                          n_jobs=-1, 
                          scoring='f1', 
                          cv = 3)

model_lr.fit(X_train, y_train)

display(model_lr.best_score_)

display(model_lr.best_params_)

best_model_lr = model_lr.best_estimator_

0.7656615546617725

{'C': 9}

CPU times: user 2min 30s, sys: 4min 24s, total: 6min 55s
Wall time: 6min 55s


**RandomForestClassifier**

In [14]:
%%time

param_grid = {'n_estimators': range(1, 101), 
              'max_depth': range(1, 11)}


estimator = RandomForestClassifier(random_state=12345, 
                                   n_jobs=-1,
                                   verbose = False, 
                                   class_weight='balanced')

model_rfc = RandomizedSearchCV(estimator, 
                          param_grid,
                          verbose = False,
                          n_jobs=-1, 
                          scoring='f1', 
                          cv = 3)

model_rfc.fit(X_train, y_train)

display(model_rfc.best_score_)

display(model_rfc.best_params_)

best_model_rfc = model_rfc.best_estimator_

0.36166749705173623

{'n_estimators': 73, 'max_depth': 9}

CPU times: user 1min 11s, sys: 1.22 s, total: 1min 12s
Wall time: 1min 12s


**DecisionTreeClassifier**

In [15]:
%%time

param_grid = {'max_depth': range(1, 11)}


estimator = DecisionTreeClassifier(random_state=12345, 
                                   class_weight='balanced')

model_dtc = RandomizedSearchCV(estimator, 
                          param_grid,
                          verbose = False,
                          n_jobs=-1, 
                          scoring='f1', 
                          cv = 3)

model_dtc.fit(X_train, y_train)

display(model_dtc.best_score_)

display(model_dtc.best_params_)

best_model_dtc = model_dtc.best_estimator_

0.54588768659647

{'max_depth': 9}

CPU times: user 2min 28s, sys: 1.72 s, total: 2min 30s
Wall time: 2min 30s


**Обучив три модели только логистическая регрессия смогла удовлетворить наши поотребности поставелнные в ТЗ. f1 > 0,75
<br>ЗЫ на бустинговых моделях отлетало ядро**

In [16]:
# тест на адекватность
preds_model_dr = np.ones(y_test.shape[0])
f1_score(y_test, preds_model_dr) 

0.18601870436008028

In [17]:
# резкльтат на тесте
preds_model_lr = best_model_lr.predict(X_test)
f1_model_lr = f1_score(y_test, preds_model_lr) 
print("f1 для  Logistic Regression на тестовой выборке:", (f1_model_lr))

f1 для  Logistic Regression на тестовой выборке: 0.761690712680345


**Как видим модель можно считать адекватной, а так же f1 на тесте больше 0.75, что не может не радовать**

## Выводы

**Во время работы были проделаны следующий вещи:
<br> 1) Рассмотрен набор данных, который был лемматизирован и векторизован
<br> 2) Классы были не сбалансированны, поэтому при обученнии прибегали к балансировке классов
<br> 3) Наиболее подходящей моделью обучения стала логичстическая регрессия
<br> 4) Адекватность модели сравнивал с Dummy моделью** 